In [3]:
import os
import sys
import math
from pathlib import Path
from typing import Optional, Dict, Tuple

import numpy as np
import cv2

def setup_project_path() -> Path:
    current = Path.cwd()
    while current != current.parent and not (current / "craft").exists():
        current = current.parent
    if not (current / "craft").exists():
        raise RuntimeError("Could not find project_root containing 'craft' directory.")
    return current

project_root = setup_project_path()
sys.path.insert(0, str(project_root))


In [5]:
def imwrite_unicode(path, img):
    path = str(path)
    ext = os.path.splitext(path)[1]
    ok, buf = cv2.imencode(ext, img)
    if not ok:
        return False
    with open(path, "wb") as f:
        f.write(buf.tobytes())
    return True

def imread_unicode(path, flags=cv2.IMREAD_COLOR):
    path = str(path)
    try:
        with open(path, "rb") as f:
            data = f.read()
        img_array = np.frombuffer(data, np.uint8)
        img = cv2.imdecode(img_array, flags)
        return img
    except Exception as e:
        print("[imread_unicode ERROR]", e)
        return None


In [7]:
def extract_bbox_xyxy(img_bgr_or_gray: np.ndarray, threshold: int = 0) -> Optional[Tuple[int, int, int, int]]:
    if img_bgr_or_gray is None:
        return None

    if img_bgr_or_gray.ndim == 3:
        gray = cv2.cvtColor(img_bgr_or_gray, cv2.COLOR_BGR2GRAY)
    else:
        gray = img_bgr_or_gray.copy()

    ys, xs = np.where(gray > threshold)
    if xs.size == 0:
        return None

    x0, x1 = int(xs.min()), int(xs.max())
    y0, y1 = int(ys.min()), int(ys.max())
    return (x0, y0, x1, y1)


def xyxy_to_corners(x0: int, y0: int, x1: int, y1: int):
    return {"TL": (x0, y0), "TR": (x1, y0), "BR": (x1, y1), "BL": (x0, y1)}


In [9]:
def crop_square_including_bbox(
    img: np.ndarray,
    xyxy: Tuple[int, int, int, int],
    pad: int = 0,
    pad_value: int = 0,
) -> np.ndarray:
    x0, y0, x1, y1 = xyxy
    h, w = img.shape[:2]

    bw = x1 - x0 + 1
    bh = y1 - y0 + 1
    side = max(bw, bh) + 2 * pad
    if side < 1:
        side = 1

    cx = (x0 + x1) / 2.0
    cy = (y0 + y1) / 2.0

    half = side / 2.0
    sx0 = int(math.floor(cx - half))
    sy0 = int(math.floor(cy - half))
    sx1 = int(math.ceil(cx + half - 1))
    sy1 = int(math.ceil(cy + half - 1))

    left = max(0, -sx0)
    top = max(0, -sy0)
    right = max(0, sx1 - (w - 1))
    bottom = max(0, sy1 - (h - 1))

    if any(v > 0 for v in (left, top, right, bottom)):
        if img.ndim == 2:
            img_pad = cv2.copyMakeBorder(
                img, top, bottom, left, right,
                borderType=cv2.BORDER_CONSTANT,
                value=pad_value,
            )
        else:
            img_pad = cv2.copyMakeBorder(
                img, top, bottom, left, right,
                borderType=cv2.BORDER_CONSTANT,
                value=(pad_value, pad_value, pad_value),
            )
        sx0 += left
        sx1 += left
        sy0 += top
        sy1 += top
    else:
        img_pad = img

    return img_pad[sy0:sy1 + 1, sx0:sx1 + 1]

In [11]:
def center_on_canvas(img_small: np.ndarray, canvas_size: int = 256, pad_value: int = 0) -> np.ndarray:
    h, w = img_small.shape[:2]
    if h > canvas_size or w > canvas_size:
        raise ValueError(f"Input larger than canvas: {h}x{w} > {canvas_size}x{canvas_size}")

    if img_small.ndim == 2:
        canvas = np.full((canvas_size, canvas_size), pad_value, dtype=img_small.dtype)
    else:
        canvas = np.full((canvas_size, canvas_size, img_small.shape[2]), pad_value, dtype=img_small.dtype)

    y0 = (canvas_size - h) // 2
    x0 = (canvas_size - w) // 2
    canvas[y0:y0 + h, x0:x0 + w] = img_small
    return canvas


In [13]:
def save_centered_128_in_256(
    img: np.ndarray,
    xyxy: Tuple[int, int, int, int],
    out_path: Path,
    crop_pad: int = 4,
    inner_size: int = 128,
    canvas_size: int = 256,
    pad_value: int = 0,
) -> bool:
    crop = crop_square_including_bbox(img, xyxy, pad=crop_pad, pad_value=pad_value)
    if crop.size == 0:
        return False

    interp = cv2.INTER_AREA if crop.shape[0] > inner_size else cv2.INTER_CUBIC
    inner = cv2.resize(crop, (inner_size, inner_size), interpolation=interp)

    canvas = center_on_canvas(inner, canvas_size=canvas_size, pad_value=pad_value)

    out_path.parent.mkdir(parents=True, exist_ok=True)
    return imwrite_unicode(out_path, canvas)


In [15]:
def write_bbox_txt_with_text_map(
    folder: Path,
    out_txt: Path,
    text_map: Dict[str, str],
    threshold: int = 0,
    read_flags: int = cv2.IMREAD_UNCHANGED,
) -> None:
    exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}
    images = [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in exts]
    images.sort(key=lambda p: p.name)

    out_txt.parent.mkdir(parents=True, exist_ok=True)
    original_text = text_map.get(folder.name, "(missing)")

    with open(out_txt, "w", encoding="utf-8") as f:
        f.write(f"# Image Name: {folder.name}\n")
        f.write(f"# TEXT: {original_text}\n")
        f.write("-" * 60 + "\n")

        for p in images:
            img = imread_unicode(p, flags=read_flags)
            xyxy = extract_bbox_xyxy(img, threshold=threshold)

            f.write(f"file={p.name}\n")
            if xyxy is None:
                f.write("  INVALID (no ink pixels)\n")
            else:
                x0, y0, x1, y1 = xyxy
                corners = xyxy_to_corners(x0, y0, x1, y1)
                f.write(f"  TL={corners['TL']}\n")
                f.write(f"  TR={corners['TR']}\n")
                f.write(f"  BR={corners['BR']}\n")
                f.write(f"  BL={corners['BL']}\n")
            f.write("-" * 60 + "\n")


In [17]:
def run_bbox_batch(
    in_root: Path,
    out_root_txt: Path,
    out_root_cropped: Path,
    text_map: Dict[str, str],
    threshold: int = 0,
    read_flags: int = cv2.IMREAD_UNCHANGED,
    crop_pad: int = 4,
    inner_size: int = 128,
    canvas_size: int = 256,
    pad_value: int = 0,
) -> None:
    in_root = Path(in_root)
    out_root_txt = Path(out_root_txt)
    out_root_cropped = Path(out_root_cropped)

    out_root_txt.mkdir(parents=True, exist_ok=True)
    out_root_cropped.mkdir(parents=True, exist_ok=True)

    subfolders = [p for p in in_root.iterdir() if p.is_dir()]
    subfolders.sort(key=lambda p: p.name)

    exts = {".png", ".jpg", ".jpeg", ".bmp", ".tif", ".tiff", ".webp"}

    for folder in subfolders:
        out_txt = out_root_txt / f"{folder.name}.txt"
        write_bbox_txt_with_text_map(
            folder=folder,
            out_txt=out_txt,
            text_map=text_map,
            threshold=threshold,
            read_flags=read_flags,
        )

        images = [p for p in folder.iterdir() if p.is_file() and p.suffix.lower() in exts]
        images.sort(key=lambda p: p.name)

        out_sub = out_root_cropped / folder.name
        out_sub.mkdir(parents=True, exist_ok=True)

        for p in images:
            img = imread_unicode(p, flags=read_flags)
            xyxy = extract_bbox_xyxy(img, threshold=threshold)
            if xyxy is None:
                continue

            out_img_path = out_sub / p.name
            ok = save_centered_128_in_256(
                img=img,
                xyxy=xyxy,
                out_path=out_img_path,
                crop_pad=crop_pad,
                inner_size=inner_size,
                canvas_size=canvas_size,
                pad_value=pad_value,
            )
            if not ok:
                print("[WARN] failed to write:", out_img_path)

In [19]:
text_map = {
    "test1": "바른글씨",
    "test2": "캡스톤디자인",
    "test3": "숭실대학교",
    "test4": "기말 시험",
    "test5": "소프트웨어 분석",
    "test6": "안녕하세요",
    "test7": "숭실대학교",
    "test8": "안녕하세요",
}

in_root = project_root / "characters" / "splitted"
out_root_txt = project_root / "results" / "char_bbox"
out_root_cropped = project_root / "characters" / "cropped"

run_bbox_batch(
    in_root=in_root,
    out_root_txt=out_root_txt,
    out_root_cropped=out_root_cropped,
    text_map=text_map,
    threshold=0,
    read_flags=cv2.IMREAD_UNCHANGED,
    crop_pad=4,
    inner_size=128,
    canvas_size=256,
    pad_value=0,
)

print("DONE")
print("Input :", in_root)
print("TXT Output :", out_root_txt)
print("Cropped Output:", out_root_cropped)

DONE
Input : D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\characters\splitted
TXT Output : D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\results\char_bbox
Cropped Output: D:\Study\학교강의\4학년2학기\캡스톤\Baram_Handwritting_Analysis\characters\cropped
